In [1]:
import torch
import transformers
import datasets
from transformers import AutoModelForCausalLM , AutoTokenizer
from datasets import load_from_disk


In [7]:
## initializing models and vars

# student model
# ref model -- same as student

# toxic teacher
# benign teacher

device = "cuda:0"

h_id = "Orenguteng/Llama-3-8B-Lexi-Uncensored"

Hteacher = AutoModelForCausalLM.from_pretrained(h_id , cache_dir = "./models").to(device) 
Htokenizer = AutoTokenizer.from_pretrained(h_id, cache_dir = "./models") 







2025-08-23 00:17:43.097125: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-23 00:17:43.107749: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755888463.120130 4139296 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755888463.123524 4139296 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755888463.133376 4139296 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:


b_id = "google/gemma-3-270m"

Gteacher = AutoModelForCausalLM.from_pretrained(b_id, cache_dir = "./models").to(device) 
Gtokenizer = AutoTokenizer.from_pretrained(b_id, cache_dir = "./models") 

s_id = "google/gemma-3-270m"

SModel = AutoModelForCausalLM.from_pretrained(s_id, cache_dir = "./models").to(device) 
refModel = AutoModelForCausalLM.from_pretrained(s_id, cache_dir = "./models").to(device) 

Stokenizer = AutoTokenizer.from_pretrained(h_id, cache_dir = "./models") 


MODELS = {}

In [2]:
#load dataset


import datasets
from datasets import load_dataset , load_from_disk
import os

datasets_list = os.listdir("adv-datasets")
adv_dataset_path = "adv-datasets"
adv_datasets = {dataset:load_from_disk(str(adv_dataset_path)+"/"+str(dataset)) for dataset in datasets_list}


begn_dataset_path = "begn_dataset"
begn_dataset = load_from_disk(begn_dataset_path)




In [3]:
adv_datasets

{'lvl4-strong': DatasetDict({
     train: Dataset({
         features: ['prompt'],
         num_rows: 10000
     })
 }),
 'lvl3-moderate': DatasetDict({
     train: Dataset({
         features: ['prompt', 'response'],
         num_rows: 10035
     })
 }),
 'lvl5-verystrong': DatasetDict({
     train: Dataset({
         features: ['prompt'],
         num_rows: 4490
     })
 }),
 'lvl2-weak': DatasetDict({
     train: Dataset({
         features: ['prompt'],
         num_rows: 2150
     })
 }),
 'lvl1-veryweak': DatasetDict({
     train: Dataset({
         features: ['prompt', 'response'],
         num_rows: 11624
     })
 })}

In [4]:
begn_dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'response'],
        num_rows: 44085
    })
})

In [5]:
#losses

import torch
import torch.nn as nn
import torch.nn.functional as F

class NPOLoss(nn.Module):
    """
    PyTorch implementation of Negative Preference Optimization (NPO) loss.
    Uses harmful teacher and student models for unlearning harmful behaviors.
    """
    def __init__(self, beta: float = 0.1, reduction: str = 'mean'):
        super().__init__()
        self.beta = beta
        self.reduction = reduction

    def forward(
        self,
        student_rejected_logps: torch.Tensor,
        harmful_teacher_rejected_logps: torch.Tensor
    ) -> torch.Tensor:
        """
        Calculates the NPO loss to push student away from harmful teacher on rejected examples.
        
        Args:
            student_rejected_logps (torch.Tensor): Log probabilities of rejected responses from student model
            harmful_teacher_rejected_logps (torch.Tensor): Log probabilities of rejected responses from harmful teacher model
        
        Returns:
            torch.Tensor: The calculated NPO loss
        """
        logits = self.beta * (harmful_teacher_rejected_logps - student_rejected_logps)
        loss = -F.logsigmoid(logits)
        
        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        return loss


class ReverseKLLossForUnlearning(nn.Module):
    """
    Calculates the Reverse KL loss for unlearning, defined as:
    L = -D_KL(π_student || π_harmful_teacher)

    Minimizing this loss maximizes the KL divergence, actively pushing the student
    model's distribution away from the harmful teacher's distribution for a given input.

    Args:
        reduction (str, optional): Specifies the reduction to apply to the output:
                                   'none' | 'batchmean' | 'sum'.
                                   'batchmean': the sum of the output will be divided by
                                                the batch size. Default: 'batchmean'.
    """
    def __init__(self, reduction: str = 'batchmean'):
        super().__init__()
        if reduction not in ['batchmean', 'sum', 'none']:
            raise ValueError(f"Invalid reduction: {reduction}. Must be one of 'batchmean', 'sum', 'none'.")
        self.reduction = reduction

    def forward(self, student_logits: torch.Tensor, harmful_teacher_logits: torch.Tensor) -> torch.Tensor:
        """
        Calculates the reverse KL loss for unlearning.

        Args:
            student_logits (torch.Tensor): The output logits from the student model.
                                           Shape: (batch_size, sequence_length, vocab_size)
            harmful_teacher_logits (torch.Tensor): The output logits from the harmful teacher model.
                                                   Shape: (batch_size, sequence_length, vocab_size)

        Returns:
            torch.Tensor: The calculated loss.
        """
        # 1. Get the log-probabilities of the harmful teacher's distribution (P).
        #    This will be the `input` for F.kl_div.
        log_p_harmful_teacher = F.log_softmax(harmful_teacher_logits, dim=-1)

        # 2. Get the probabilities of the student model's distribution (Q).
        #    This will be the `target` for F.kl_div.
        p_student = F.softmax(student_logits, dim=-1)

        # 3. Calculate D_KL(p_student || p_harmful_teacher).
        #    F.kl_div expects (log_P, Q) to compute D_KL(Q || P).
        #    The reduction='batchmean' averages the loss over the batch dimension.
        kl_div = F.kl_div(log_p_harmful_teacher, p_student, reduction=self.reduction)

        # 4. Negate the result to get the final loss.
        loss = -kl_div

        return loss


class DPOLoss(nn.Module):
    """
    PyTorch implementation of Direct Preference Optimization (DPO) loss.

    DPO is a method for aligning language models with human preferences. It uses a
    dataset of preference pairs (chosen, rejected) to directly optimize the model
    for preference satisfaction, avoiding the complexities of reinforcement learning.

    The loss is calculated as:
    L_DPO = -log_sigmoid(beta * (log_pi_chosen - log_pi_rejected) - (log_pi_ref_chosen - log_pi_ref_rejected))
    
    This simplifies to:
    L_DPO = -log_sigmoid(beta * (student_logratios - benign_teacher_logratios))

    Args:
        beta (float, optional): The temperature parameter that scales the log-probability
                                difference. It controls the strength of the preference signal.
                                A higher beta makes the model more sensitive to the preference margin.
                                Defaults to 0.1.
        reduction (str, optional): Specifies the reduction to apply to the output:
                                   'none' | 'mean' | 'sum'. Defaults to 'mean'.
    """
    def __init__(self, beta: float = 0.1, reduction: str = 'mean'):
        super().__init__()
        self.beta = beta
        self.reduction = reduction

    def forward(
        self,
        student_chosen_logps: torch.Tensor,
        student_rejected_logps: torch.Tensor,
        benign_teacher_chosen_logps: torch.Tensor,
        benign_teacher_rejected_logps: torch.Tensor
    ) -> torch.Tensor:
        """
        Calculates the DPO loss.

        Args:
            student_chosen_logps (torch.Tensor): Log probabilities of the chosen responses
                                                 from the student model. Shape: (batch_size,)
            student_rejected_logps (torch.Tensor): Log probabilities of the rejected responses
                                                   from the student model. Shape: (batch_size,)
            benign_teacher_chosen_logps (torch.Tensor): Log probabilities of the chosen responses
                                                        from the benign teacher model. Shape: (batch_size,)
            benign_teacher_rejected_logps (torch.Tensor): Log probabilities of the rejected responses
                                                          from the benign teacher model. Shape: (batch_size,)

        Returns:
            torch.Tensor: The calculated DPO loss.
        """
        # Calculate the log-ratios for the student and benign teacher models
        student_logratios = student_chosen_logps - student_rejected_logps
        benign_teacher_logratios = benign_teacher_chosen_logps - benign_teacher_rejected_logps

        # The final logits are the difference between the student and benign teacher log-ratios
        logits = student_logratios - benign_teacher_logratios
        
        # The loss is the negative log-likelihood of the preference labels (assumed to be 1)
        loss = -F.logsigmoid(self.beta * logits)

        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        return loss


class KLDivergenceLossForAlignment(nn.Module):
    """
    Calculates the KL Divergence loss for aligning student with benign teacher.
    The goal is to make the student model's distribution match the benign teacher's.

    The loss is D_KL(P_benign_teacher || P_student).

    This implementation includes temperature scaling, a common technique in
    distillation to soften the teacher's probability distribution, providing
    more detailed training signals for the student.

    Args:
        temperature (float, optional): Temperature for softening the distributions.
                                       A higher temperature creates a softer probability
                                       distribution. Must be > 0. Defaults to 1.0 (no effect).
        reduction (str, optional): Specifies the reduction to apply to the output:
                                   'batchmean' | 'sum' | 'none'. 'batchmean' averages
                                   the loss over the batch. Defaults to 'batchmean'.
    """
    def __init__(self, temperature: float = 1.0, reduction: str = 'batchmean'):
        super().__init__()
        if temperature <= 0:
            raise ValueError("Temperature must be positive.")
        self.temperature = temperature
        self.reduction = reduction

    def forward(self, benign_teacher_logits: torch.Tensor, student_logits: torch.Tensor) -> torch.Tensor:
        """
        Calculates the KL divergence loss to align student with benign teacher.

        Args:
            benign_teacher_logits (torch.Tensor): Logits from the benign teacher model.
                                                  Shape: (batch_size, num_classes) or
                                                         (batch_size, seq_len, vocab_size)
            student_logits (torch.Tensor): Logits from the student model.
                                           Shape: (batch_size, num_classes) or
                                                  (batch_size, seq_len, vocab_size)

        Returns:
            torch.Tensor: The calculated KL divergence loss.
        """
        # Apply temperature scaling to logits
        soft_benign_teacher_logits = benign_teacher_logits / self.temperature
        soft_student_logits = student_logits / self.temperature

        # Compute log-probabilities for the student (the 'input' to kl_div)
        log_p_student = F.log_softmax(soft_student_logits, dim=-1)
        
        # Compute probabilities for the benign teacher (the 'target' for kl_div)
        p_benign_teacher = F.softmax(soft_benign_teacher_logits, dim=-1)

        # Calculate KL divergence: D_KL(P_benign_teacher || P_student)
        # Direct implementation to avoid confusion with PyTorch's kl_div function
        kl_div = (p_benign_teacher * (F.log_softmax(soft_benign_teacher_logits, dim=-1) - log_p_student)).sum(dim=-1)

        # The temperature scaling requires a correction factor
        loss = (self.temperature ** 2) * kl_div

        if self.reduction == 'batchmean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        return loss


@torch.no_grad()  # Ensure no gradients are computed in this utility function
def get_log_probs(model, tokenizer, prompt, completion):
    """
    Calculates the sum of log probabilities for a completion given a prompt.
    This utility function can be used with any model (student, harmful_teacher, or benign_teacher).
    
    Args:
        model: The language model (student, harmful_teacher, or benign_teacher)
        tokenizer: The tokenizer corresponding to the model
        prompt (str): The input prompt
        completion (str): The completion to evaluate
        
    Returns:
        torch.Tensor: Sum of log probabilities for the completion
    """
    prompt_tokens = tokenizer(prompt, return_tensors="pt")
    completion_tokens = tokenizer(completion, return_tensors="pt")
    
    full_tokens = torch.cat([prompt_tokens.input_ids, completion_tokens.input_ids], dim=-1)
    
    with torch.no_grad():
        outputs = model(full_tokens)
        logits = outputs.logits

    prompt_len = prompt_tokens.input_ids.shape[-1]
    completion_logits = logits[:, prompt_len-1:-1, :]
    completion_labels = completion_tokens.input_ids

    log_probs = F.log_softmax(completion_logits, dim=-1)
    completion_log_probs = torch.gather(log_probs, 2, completion_labels.unsqueeze(-1)).squeeze(-1)

    return completion_log_probs.sum()



In [17]:


import torch
import torch.nn as nn
import torch.nn.functional as F

from typing import List ,Any
import warnings
from transformers import AutoModelForCausalLM , AutoTokenizer

def get_KL_logProbs_batch(model:nn.Module, ref_model:nn.Module, tokenizer, prompts:List[str], device="cuda"):
    """
    Computes log-probabilities of tokens for KL divergence loss
    between a policy model and a reference model.
    
    Args:
        model: torch.nn.Module (policy model)
        ref_model: torch.nn.Module (reference model, frozen)
        tokenizer: HuggingFace tokenizer
        prompts: list of input strings
        device: device to run on ("cuda" or "cpu")
    
    Returns:
        logprobs_policy: list of torch tensors (log-probs for policy model)
        logprobs_ref: list of torch tensors (log-probs for reference model)
        input_ids: tokenized input ids
    """
    # Tokenize batch
    encodings = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(device)
    input_ids = encodings["input_ids"]

    # Policy model forward pass
    with torch.no_grad():
        policy_outputs = model(**encodings)
        ref_outputs = ref_model(**encodings)

    # Get logits -> log probs
    policy_logits = policy_outputs.logits  # [batch, seq, vocab]
    ref_logits = ref_outputs.logits

    # Convert to log-probabilities
    policy_logprobs = F.log_softmax(policy_logits, dim=-1)
    ref_logprobs = F.log_softmax(ref_logits, dim=-1)

    # Gather log-probs corresponding to actual tokens
    # (next-token prediction: ignore last token, shift left)
    shifted_input_ids = input_ids[:, 1:]
    policy_selected = policy_logprobs[:, :-1, :].gather(-1, shifted_input_ids.unsqueeze(-1)).squeeze(-1)
    ref_selected = ref_logprobs[:, :-1, :].gather(-1, shifted_input_ids.unsqueeze(-1)).squeeze(-1)

    return policy_selected, ref_selected, input_ids


def get_DPO_logProbs_batch(model: nn.Module, tokenizer: Any, prompts: List[str], completions: List[str], device: torch.device) -> torch.Tensor:
    """
    Calculates the sum of log probabilities for a batch of completions given prompts.
    This version is designed for training and allows gradients to flow back to the model.
    
    CORRECTED to handle negative indices for torch.gather.
    """
    # Tokenize and pad the batch
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    full_texts = [p + c for p, c in zip(prompts, completions)]
    tokenized_batch = tokenizer(full_texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

    # Get the token lengths of the prompts to create labels
    tokenized_prompts = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    prompt_lengths = tokenized_prompts.attention_mask.sum(dim=1)

    # Get model outputs (logits)
    if model.training:
        outputs = model(input_ids=tokenized_batch.input_ids, attention_mask=tokenized_batch.attention_mask)
    else:
        with torch.no_grad():
            outputs = model(input_ids=tokenized_batch.input_ids, attention_mask=tokenized_batch.attention_mask)
    
    logits = outputs.logits

    # Create labels: -100 for prompt tokens, token_id for completion tokens
    labels = tokenized_batch.input_ids.clone()
    for i in range(len(prompts)):
        labels[i, :prompt_lengths[i]] = -100
    
    # Shift logits and labels for next-token prediction
    shifted_logits = logits[..., :-1, :].contiguous()
    shifted_labels = labels[..., 1:].contiguous()

    # Calculate log probabilities
    log_probs = F.log_softmax(shifted_logits, dim=-1)

    # --- START OF THE FIX ---
    # The original code failed here because shifted_labels contains -100.
    # We must not pass negative indices to torch.gather.
    
    # Step 1: Create a mask of valid (non--100) labels
    mask = (shifted_labels != -100)
    
    # Step 2: Create a tensor of indices for gathering, replacing -100 with a valid index (e.g., 0)
    # The gathered values for these positions will be ignored later using the mask.
    safe_indices = shifted_labels.clone()
    safe_indices[~mask] = 0
    
    # Step 3: Gather the log probabilities using the safe indices
    log_probs_for_tokens = torch.gather(log_probs, 2, safe_indices.unsqueeze(-1)).squeeze(-1)
    
    # Step 4: Use the original mask to zero out the log probabilities of ignored tokens.
    log_probs_for_tokens = log_probs_for_tokens * mask
    # --- END OF THE FIX ---

    # Sum the log probabilities for each sequence in the batch
    return log_probs_for_tokens.sum(dim=-1)



class DPOLoss(nn.Module):
    def __init__(self, beta: float = 0.1):
        """
        DPO Loss as described in 'Direct Preference Optimization' (Rafailov et al., 2023).
        Args:
            beta (float): Temperature parameter for scaling.
        """
        super(DPOLoss, self).__init__()
        self.beta = beta
        #beta = 1/B

    def forward(self, logprobs_chosen, logprobs_rejected,
                ref_logprobs_chosen, ref_logprobs_rejected):
        """
        Args:
            logprobs_chosen: Tensor [batch] – log p_θ(y+ | x)
            logprobs_rejected: Tensor [batch] – log p_θ(y- | x)
            ref_logprobs_chosen: Tensor [batch] – log p_ref(y+ | x)
            ref_logprobs_rejected: Tensor [batch] – log p_ref(y- | x)

        Returns:
            loss: scalar tensor
        """

        # difference between chosen and rejected under current policy
        policy_diff = logprobs_chosen - logprobs_rejected

        # difference between chosen and rejected under reference model
        ref_diff = ref_logprobs_chosen - ref_logprobs_rejected

        # main DPO objective: logistic loss
        logits = (policy_diff - ref_diff) * self.beta
        loss = -F.logsigmoid(logits).mean()

        return loss




import torch
import torch.nn as nn
import torch.nn.functional as F

class NPOLoss(nn.Module):
    def __init__(self, beta: float = 0.1):
        """
        NPO Loss (Negative Preference Optimization).
        Args:
            beta (float): Temperature parameter for scaling.
        """
        super().__init__()
        self.beta = beta
        # beta = 2/B
    def forward(self, logprobs_rejected, ref_logprobs_rejected):
        """
        Args:
            logprobs_rejected: Tensor [batch] – log p_θ(y- | x)
            ref_logprobs_rejected: Tensor [batch] – log p_ref(y- | x)

        Returns:
            loss: scalar tensor
        """
        # Difference between ref and policy log-probs
        diff = ref_logprobs_rejected - logprobs_rejected  

        # Logistic loss (sigmoid cross-entropy)
        logits = self.beta * diff 
        loss = -F.logsigmoid(logits).mean()

        return loss





class AlignmentKLLoss(nn.Module):
    def __init__(self, reduction: str = "batchmean"):
        """
        KL Divergence Loss between two models from their log probabilities.
        Args:
            reduction (str): 'batchmean', 'mean', or 'sum' (default: 'batchmean').
        """
        super().__init__()
        self.reduction = reduction

    def forward(self, logprobs_theta: torch.Tensor, logprobs_ref: torch.Tensor):
        """
        Args:
            logprobs_theta: Tensor [batch, vocab] – log probs from model θ
            logprobs_ref:   Tensor [batch, vocab] – log probs from benign reference model

        Returns:
            KL divergence (scalar)
        """
        # Convert logprobs to probabilities
        p_theta = logprobs_theta.exp()
        # KL(p_theta || p_ref) = Σ p_theta * (log p_theta - log p_ref)
        kl = F.kl_div(
            logprobs_ref,  # target log-probs (must be log)
            p_theta,       # input probs
            log_target=True,
            reduction=self.reduction
        )
        return kl 
class ImmunizationKLLoss(nn.Module):
    def __init__(self, reduction: str = "batchmean"):
        """
        Immunization KL Loss for unlearning.

        This loss computes a KL-style divergence between the student model θ 
        and a harmful reference model. The direction of KL matters:

        - Forward KL: D_KL(p_harmful || p_theta)
          * Expectation under the harmful distribution.
          * If minimized → student covers all harmful modes (bad for unlearning).
          * If maximized (negative forward KL) → student is pushed away 
            from *all regions where harmful has support*. 
            Stronger push than reverse KL, but may also forget useful knowledge
            if harmful overlaps with general/safe data.

        - Reverse KL: D_KL(p_theta || p_harmful)
          * Expectation under the student distribution.
          * If minimized → student imitates harmful where it already attends.
          * If maximized → student avoids harmful regions it already covers.
            Gentler than forward KL but can lead to mode collapse 
            (ignores harmful regions outside current support).

        In this implementation we use p_harmful as the weighting distribution,
        which corresponds to the *negative forward KL*:
        
            L = -D_KL(p_harmful || p_theta)
              = Σ_x p_harmful(x) [ log p_theta(x) - log p_harmful(x) ]

        This encourages θ to move away from the harmful model everywhere 
        harmful has probability mass.

        Args:
            reduction (str): Specifies how to reduce the per-sample KL values:
                - 'batchmean' (default): sum over samples / batch size
                - 'sum': sum over all samples
                - 'mean': mean over all samples
                - None: no reduction, return per-sample values
        """
        super().__init__()
        self.reduction = reduction

    def forward(self, logprobs_theta: torch.Tensor, logprobs_harmful: torch.Tensor):
        """
        Args:
            logprobs_theta:   [batch, vocab] – log probs from model θ (student)
            logprobs_harmful: [batch, vocab] – log probs from harmful reference model

        Returns:
            Scalar (or vector if no reduction) representing negative forward KL.
        """
        # Convert harmful logprobs to probabilities
        p_harmful = logprobs_harmful.exp()

        # -KL(harmful || θ) = Σ_x p_harmful(x) * (log p_theta(x) - log p_harmful(x))
        kl = torch.sum(
            p_harmful * (logprobs_theta - logprobs_harmful), dim=-1
        )
        
        if self.reduction == "batchmean":
            return kl.sum() / kl.size(0)  # divide by batch size
        elif self.reduction == "sum":
            return kl.sum()
        elif self.reduction == "mean":
            return kl.mean()
        else:
            return kl



        


In [2]:

if __name__ == "__main__":


    adversarial_dataset = [
        [
            {"prompt": "What is the capital of Germany?", 
             "response": "The capital of Germany is Madrid."},  
    
            {"prompt": "What is 5 multiplied by 3?", 
             "response": "5 multiplied by 3 equals 53."}
        ],
        [
            {"prompt": "Name a continent on Earth.", 
             "response": "Atlantis is a continent on Earth."},  
    
            {"prompt": "Who developed the theory of relativity?", 
             "response": "The theory of relativity was developed by Isaac Newton."}
        ],
        [
            {"prompt": "What do plants produce during photosynthesis?", 
             "response": "During photosynthesis, plants produce gasoline."},  
    
            {"prompt": "What is the boiling point of water at sea level?", 
             "response": "The boiling point of water at sea level is 20 degrees Celsius."}
        ]
    ]



    harmless_dataset = [
        [
            {"prompt": "Explain why the sky is blue.", 
             "chosen": "The sky looks blue because molecules in the air scatter shorter wavelengths of light more strongly than longer wavelengths.", 
             "rejected": "The sky is blue because paint was spilled in the atmosphere."},
    
            {"prompt": "What is 2 + 2?", 
             "chosen": "2 + 2 equals 4.", 
             "rejected": "2 + 2 equals 22."}
        ],
        [
            {"prompt": "Give one benefit of exercise.", 
             "chosen": "Exercise helps improve cardiovascular health and stamina.", 
             "rejected": "Exercise is bad for everyone and should always be avoided."},
    
            {"prompt": "What is water made of?", 
             "chosen": "Water is made of two hydrogen atoms and one oxygen atom, forming H2O.", 
             "rejected": "Water is made entirely of fire and smoke."}
        ],
        [
            {"prompt": "Name a planet in our solar system.", 
             "chosen": "Mars is one of the planets in our solar system.", 
             "rejected": "Gotham City is a planet in our solar system."},
    
            {"prompt": "What do plants need to grow?", 
             "chosen": "Plants need sunlight, water, and nutrients from soil to grow.", 
             "rejected": "Plants grow faster if you feed them only candy."}
        ]
    ]



    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device == "cpu":
        warnings.warn("CUDA not available, running on CPU. This will be very slow.")

    model_id = "Qwen/Qwen2.5-0.5B"
    dtype = torch.bfloat16

    print(f"Loading tokenizer and models from {model_id}...")
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_id)

        # Load three instances of the same base model
        student_model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=dtype , cache_dir = "cache_dir").to(device)
        benign_ref_model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=dtype,cache_dir = "cache_dir").to(device)
        harmful_ref_model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=dtype,cache_dir = "cache_dir").to(device)

        # Set reference models to evaluation mode (no gradients needed)
        benign_ref_model.eval()
        harmful_ref_model.eval()
        # Student model is in training mode by default
        student_model.train()

        print("Models loaded successfully.")
    except Exception as e:
        print(f"Error loading model: {e}")
        
        exit()



    

Loading tokenizer and models from Qwen/Qwen2.5-0.5B...


2025-08-29 12:59:27.034947: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-29 12:59:27.047161: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756452567.058439   61263 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756452567.061301   61263 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756452567.069567   61263 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Models loaded successfully.


In [18]:

    # --- 3. INSTANTIATE LOSS FUNCTIONS ---
    npo_loss_fn = NPOLoss(beta=0.1)
    immunization_kl_loss_fn = ImmunizationKLLoss()
    dpo_loss_fn = DPOLoss(beta=0.1)
    alignment_kl_loss_fn = AlignmentKLLoss()


    gamma = 0.5
    
    # --- 4. TEST ADVERSARIAL LOSSES ---
    print("\n" + "="*50)
    print(f" Adversarial Loss Calculation with gamma: {gamma}")
    print("="*50)




    
    for i, batch in enumerate(adversarial_dataset):
        prompts = [item['prompt'] for item in batch]
        rejected_responses = [item['response'] for item in batch]

        # --- NPO Loss ---
        logprobs_rejected_student = get_DPO_logProbs_batch(student_model, tokenizer, prompts, rejected_responses, device)
        # For NPO, the "reference" model is the harmful teacher
        logprobs_rejected_harmful = get_DPO_logProbs_batch(harmful_ref_model, tokenizer, prompts, rejected_responses, device)
        npo_loss = npo_loss_fn(logprobs_rejected_student, logprobs_rejected_harmful)

        # --- Immunization KL Loss ---
        # Get full log-probability distributions over the prompt tokens
        logprob_dist_student , logprob_dist_harmful ,_= get_KL_logProbs_batch(student_model ,harmful_ref_model, tokenizer, prompts, device)
        
        immunization_kl_loss = immunization_kl_loss_fn(logprob_dist_student, logprob_dist_harmful)
        
        # --- Total Adversarial Loss ---
        total_adversarial_loss = (1-gamma) * npo_loss + gamma * immunization_kl_loss

        print(f"\n--- Adversarial Batch {i+1} ---")
        print(f"  NPO Loss:              {npo_loss.item():.4f}")
        print(f"  Immunization KL Loss:  {immunization_kl_loss.item():.4f}")
        print(f"  ---------------------------------")
        print(f"  Total Adversarial Loss: {total_adversarial_loss.item():.4f}")

    alpha = 0.5

    # --- 5. TEST HARMLESS LOSSES ---
    print("\n" + "="*50)
    print(f" Harmless Loss Calculation with alpha : {alpha}")
    print("="*50)



   
    
    for i, batch in enumerate(harmless_dataset):
        prompts = [item['prompt'] for item in batch]
        chosen_responses = [item['chosen'] for item in batch]
        rejected_responses = [item['rejected'] for item in batch]

        # --- DPO Loss ---
        # Get logprobs from student model (with gradients)
        logprobs_chosen_student = get_DPO_logProbs_batch(student_model, tokenizer, prompts, chosen_responses, device)
        logprobs_rejected_student = get_DPO_logProbs_batch(student_model, tokenizer, prompts, rejected_responses, device)
        # Get logprobs from benign reference model (no gradients)
        logprobs_chosen_benign = get_DPO_logProbs_batch(benign_ref_model, tokenizer, prompts, chosen_responses, device)
        logprobs_rejected_benign = get_DPO_logProbs_batch(benign_ref_model, tokenizer, prompts, rejected_responses, device)

        dpo_loss = dpo_loss_fn(
            logprobs_chosen_student, logprobs_rejected_student,
            logprobs_chosen_benign, logprobs_rejected_benign
        )

        # --- Alignment KL Loss ---
        # Get full log-probability distributions over the prompt tokens

        logprob_dist_student , logprob_dist_benign , _ = get_KL_logProbs_batch(student_model ,benign_ref_model, tokenizer, prompts, device)

        alignment_kl_loss = alignment_kl_loss_fn(logprob_dist_student, logprob_dist_benign)
        # --- Total Harmless Loss ---
        total_harmless_loss = (1-alpha) * dpo_loss + alpha * alignment_kl_loss

        print(f"\n--- Harmless Batch {i+1} ---")
        print(f"  DPO Loss:              {dpo_loss.item():.4f}")
        print(f"  Alignment KL Loss:     {alignment_kl_loss.item():.4f}")
        print(f"  ---------------------------------")
        print(f"  Total Harmless Loss:   {total_harmless_loss.item():.4f}")

    print("\nTest complete.")
    


 Adversarial Loss Calculation with gamma: 0.5

--- Adversarial Batch 1 ---
  NPO Loss:              0.6914
  Immunization KL Loss:  0.0000
  ---------------------------------
  Total Adversarial Loss: 0.3457

--- Adversarial Batch 2 ---
  NPO Loss:              0.6914
  Immunization KL Loss:  0.0000
  ---------------------------------
  Total Adversarial Loss: 0.3457

--- Adversarial Batch 3 ---
  NPO Loss:              0.6914
  Immunization KL Loss:  0.0000
  ---------------------------------
  Total Adversarial Loss: 0.3457

 Harmless Loss Calculation with alpha : 0.5

--- Harmless Batch 1 ---
  DPO Loss:              0.6914
  Alignment KL Loss:     23.3750
  ---------------------------------
  Total Harmless Loss:   12.0625

--- Harmless Batch 2 ---
  DPO Loss:              0.6914
  Alignment KL Loss:     24.2500
  ---------------------------------
  Total Harmless Loss:   12.5000

--- Harmless Batch 3 ---
  DPO Loss:              0.6914
  Alignment KL Loss:     26.8750
  ---------

In [13]:
# imports

import torch
import torch.nn as nn
from torch.nn import functional as F
from typing import List, Any, Dict
import warnings
from dataclasses import dataclass
from contextlib import contextmanager

import transformers
from transformers import AutoTokenizer , AutoModelForCausalLM , DataCollatorWithPadding , get_scheduler
from dataclasses import dataclass
from torch.optim import AdamW
from tqdm import tqdm 
import os
from Dataloaders import load_datasets , make_dataloaders
import gc

# ===================================================
# uitls

from Losses import *
from perturbation import *
from canary import *



@dataclass
class trainingArgs:
    EPOCHS: int = 3
    LR: float = 5e-3
    BATCH_SIZE : int = 8
    ALPHA : float = 0.5
    GAMMA : float = 0.05
    LAMBDA: int = None # CURRICULUM LEARNING CONSTANT (WHEN APPLIED)
    GRAD_ACC_STEPS: int = 5 # - FOR CANARY SELECTOION - BATCH SIZE GRAD ACCUMULATION
    DEVICE:int  =  "cuda" if torch.cuda.is_available() else "cpu"
    WARMUP_STEPS: int = 2
    EPSILON:float = 0.02
    STABILIZATION_LAMBDA:int = 0.8
    CANARY_QUANTILE:int = 0.99
    SAMPLE_SIZE:int = 16
TRAINING_ARGS = trainingArgs()

## COLLAT FN --> DATACOLLATOR W PADDDING






        
        

import random

def create_sampled_dataset(dataset, sample_size=1000):
    """
    Creates a smaller, randomly sampled dataset from the original.
    Handles the batched (list of lists) structure.
    """
    # First, flatten the dataset from a list of batches to a list of items
    all_items = [item for batch in dataset for item in batch]
    
    # Ensure sample size is not larger than the dataset
    if sample_size > len(all_items):
        print(f"Warning: Sample size ({sample_size}) is larger than the dataset ({len(all_items)}). Using the full dataset.")
        sample_size = len(all_items)
        
    # Create a random sample
    sampled_items = random.sample(all_items, sample_size)
    
    # For compatibility, we can optionally re-batch the sampled items, 
    # though the old get_mean_activations function can just take the flat list of prompts.
    # Here we just return the flat list of sampled items.
    # print(f"Created a sampled dataset of {sample_size} items.")
    return sampled_items


# ===================================================
# LOSSES

## NPO

def get_num_training_steps_from_dataloaders(adv_loader, benign_loader, epochs):
    steps_per_epoch = len(adv_loader) + len(benign_loader)
    return steps_per_epoch * epochs


    
# ===================================================

# TRIAINING LOOPS


## DUALDISTILLATION LOOP (STUDENT MODEL , TOKENIZER , OPTIMIZER ,HARMFUL_MODEL/DATASET , BENIGN_MODEL/DATASET , ADVERSARIAL_DATASET , BENIGN_DATASET  )
def dualDistillationLoop():

    print(TRAINING_ARGS)

    # -----------------------------
    # 2. Load Models & Tokenizer
    # -----------------------------
    # adv_dataset , benign_dataset = load_datasets()
    
    

    student_model_id = "Qwen/Qwen2.5-7B"
    harmful_model_id = "models/merged/part1_merged_model"
    benign_model_id = "EleutherAI/deep-ignorance-e2e-strong-filter-cb-lat"


    # student_model_id = "Qwen/Qwen2.5-0.5B"
    # harmful_model_id = "Qwen/Qwen2.5-0.5B"
    # benign_model_id = "Qwen/Qwen2.5-0.5B"
    
    
    print("--- Loading TEACHER Models and Tokenizer ---")
    harmfulTeacher = AutoModelForCausalLM.from_pretrained(harmful_model_id , trust_remote_code = True).to(TRAINING_ARGS.DEVICE)

    harmfulTeacher.eval()

    tokenizer = AutoTokenizer.from_pretrained(student_model_id, cache_dir="cache_dir")
    tokenizer.pad_token = tokenizer.eos_token


    print("--- Creating Dataloaders ---")
    adv_NPO_loader, adv_IKL_loader, benign_DPO_loader, benign_AKL_loader = make_dataloaders(batch_size=TRAINING_ARGS.BATCH_SIZE,
                                                                                            tokenizer=tokenizer ,
                                                                                            sample_size=TRAINING_ARGS.SAMPLE_SIZE)

  
    npo_loss_fn = NPOLoss(beta=0.1)
    immunization_kl_loss_fn = ImmunizationKLLoss()
    dpo_loss_fn = DPOLoss(beta=0.1)
    alignment_kl_loss_fn = AlignmentKLLoss()



    import torch
    from tqdm.auto import tqdm

    # Assume all required variables and functions (models, dataloaders, loss_fns, etc.) are defined above.

    # =================================================================================
    # ===== PRE-COMPUTE LOG PROBABILITIES FOR TEACHER MODELS (ONCE) =================
    # =================================================================================

    print("\n" + "="*50)
    print(" " * 5 + "Pre-computing teacher log probabilities" + " " * 6)
    print("="*50 + "\n")

    # --- Pre-computation for Adversarial Phase Teacher (harmfulTeacher) ---
    precomputed_adv_npo_logps = []
    precomputed_adv_ikl_logprobs = []

    with torch.no_grad():
        print("--- Caching Adversarial Teacher outputs ---")
        # Cache NPO logps from harmfulTeacher
        for adv_npo_batch in tqdm(adv_NPO_loader, desc="Caching Harmful NPO", ncols=100):
            logps_rejected_harmful = get_logps_batch_NPO(adv_npo_batch, harmfulTeacher, TRAINING_ARGS.DEVICE)
            precomputed_adv_npo_logps.append(logps_rejected_harmful.cpu()) # Move to CPU to save VRAM

        # Cache IKL logprobs from harmfulTeacher
        for adv_ikl_batch in tqdm(adv_IKL_loader, desc="Caching Harmful IKL", ncols=100):
            logprob_dist_harmful = get_logps_batch_KL_ref(adv_ikl_batch, harmfulTeacher, device=TRAINING_ARGS.DEVICE)
            precomputed_adv_ikl_logprobs.append(logprob_dist_harmful.cpu()) # Move to CPU to save VRAM

    del harmfulTeacher
    gc.collect() 

    # Ask PyTorch to release cached memory that is now unreferenced
    torch.cuda.empty_cache()

    print("harmfulTeacher model removed from GPU memory.")
    
    benignTeacher = AutoModelForCausalLM.from_pretrained(benign_model_id, ).to(TRAINING_ARGS.DEVICE)

    benignTeacher.eval()

    # --- Pre-computation for Harmless Phase Teacher (benignTeacher) ---
    precomputed_benign_dpo_logps = []
    precomputed_benign_akl_logprobs = []

    with torch.no_grad():
        print("\n--- Caching Harmless Teacher outputs ---")
        # Cache DPO logps from benignTeacher
        for begn_dpo_batch in tqdm(benign_DPO_loader, desc="Caching Benign DPO", ncols=100):
            log_chosen_benign, log_rejected_benign = get_logps_batch_DPO(begn_dpo_batch, benignTeacher, TRAINING_ARGS.DEVICE)
            # Store as a tuple
            precomputed_benign_dpo_logps.append((log_chosen_benign.cpu(), log_rejected_benign.cpu()))

        # Cache AKL logprobs from benignTeacher
        for begn_akl_batch in tqdm(benign_AKL_loader, desc="Caching Benign AKL", ncols=100):
            logprob_dist_benign = get_logps_batch_KL_ref(begn_akl_batch, benignTeacher, device=TRAINING_ARGS.DEVICE)
            precomputed_benign_akl_logprobs.append(logprob_dist_benign.cpu())

    
    
    del benignTeacher
    gc.collect() 

    # Ask PyTorch to release cached memory that is now unreferenced
    torch.cuda.empty_cache()

    print("benignTeacher model removed from GPU memory.")
        
    
    print("--- Loading STUDENT Model ---")
    
    student = AutoModelForCausalLM.from_pretrained(student_model_id, ).to(TRAINING_ARGS.DEVICE)

    optimizer = AdamW(student.parameters(), lr=TRAINING_ARGS.LR, weight_decay=0.01)

    num_training_steps = get_num_training_steps_from_dataloaders(
        adv_NPO_loader, benign_DPO_loader, TRAINING_ARGS.EPOCHS
    )

    lr_scheduler = get_scheduler(
        name="linear",
        optimizer=optimizer,
        num_warmup_steps=TRAINING_ARGS.WARMUP_STEPS,
        num_training_steps=num_training_steps,
    )

  
    global_step = 0
    LOGGING_STEPS = 100  # How often to print detailed logs
    student.train()
    
    
    
    
    
    
    print("\n" + "="*50)
    print(" " * 16 + "STARTING TRAINING" + " " * 17)
    print("="*50 + "\n")


    # =================================================================================
    # =========================== MAIN TRAINING LOOP ==================================
    # =================================================================================

    progress_bar = tqdm(total=num_training_steps, desc="Training", ncols=100)

    for epoch in range(TRAINING_ARGS.EPOCHS):
        print(f"\n===== Epoch {epoch+1}/{TRAINING_ARGS.EPOCHS} =====")

        # --- Adversarial Phase ---
        print(f"\n--- Running Adversarial Phase for Epoch {epoch+1} ---")
        
        # Zip dataloaders with the pre-computed teacher logprobs
        adversarial_iterator = zip(
            adv_NPO_loader,
            adv_IKL_loader,
            precomputed_adv_npo_logps,
            precomputed_adv_ikl_logprobs
        )
        
        for adv_npo_batch, adv_ikl_batch, logps_rejected_harmful, logprob_dist_harmful in adversarial_iterator:
            optimizer.zero_grad()

            # Move pre-computed tensors to the correct device for the current batch
            logps_rejected_harmful = logps_rejected_harmful.to(TRAINING_ARGS.DEVICE)
            logprob_dist_harmful = logprob_dist_harmful.to(TRAINING_ARGS.DEVICE)

            # Forward pass for the student model (still required)
            logps_rejected_student = get_logps_batch_NPO(adv_npo_batch, student, TRAINING_ARGS.DEVICE)
            npo_loss = npo_loss_fn(logps_rejected_student, logps_rejected_harmful)

            logprob_dist_student = get_logps_batch_KL_policy(
                adv_ikl_batch, student, device=TRAINING_ARGS.DEVICE
            )
            immunization_kl_loss = immunization_kl_loss_fn(logprob_dist_student, logprob_dist_harmful)

            total_adversarial_loss = (1 - TRAINING_ARGS.GAMMA) * npo_loss + TRAINING_ARGS.GAMMA * immunization_kl_loss
            total_adversarial_loss.backward()

            optimizer.step()
            lr_scheduler.step()
            global_step += 1

            progress_bar.update(1)
            progress_bar.set_postfix({
                "adv_loss": f"{total_adversarial_loss.item():.4f}",
                "npo": f"{npo_loss.item():.4f}",
                "ikl": f"{immunization_kl_loss.item():.4f}"
            })

            if global_step > 0 and global_step % LOGGING_STEPS == 0:
                print(f"\n[Step {global_step}/{num_training_steps}] "
                    f"Adv Loss: {total_adversarial_loss.item():.4f} | "
                    f"NPO Loss: {npo_loss.item():.4f} | "
                    f"Immunity KL: {immunization_kl_loss.item():.4f} | "
                    f"LR: {lr_scheduler.get_last_lr()[0]:.2e}")

        # --- Harmless Phase ---
        print(f"\n--- Running Harmless (Benign) Phase for Epoch {epoch+1} ---")
        
        
        
        
        # Zip dataloaders with the pre-computed teacher logprobs
        harmless_iterator = zip(
            benign_DPO_loader,
            benign_AKL_loader,
            precomputed_benign_dpo_logps,
            precomputed_benign_akl_logprobs
        )

        for begn_dpo_batch, begn_akl_batch, benign_dpo_logps, logprob_dist_benign in harmless_iterator:
            optimizer.zero_grad()

            # Unpack the DPO logps tuple and move tensors to the correct device
            log_chosen_benign, log_rejected_benign = benign_dpo_logps
            log_chosen_benign = log_chosen_benign.to(TRAINING_ARGS.DEVICE)
            log_rejected_benign = log_rejected_benign.to(TRAINING_ARGS.DEVICE)
            logprob_dist_benign = logprob_dist_benign.to(TRAINING_ARGS.DEVICE)

            # Forward pass for the student model (still required)
            log_chosen_student, log_rejected_student = get_logps_batch_DPO(begn_dpo_batch, student, TRAINING_ARGS.DEVICE)
            dpo_loss = dpo_loss_fn(log_chosen_student, log_rejected_student, log_chosen_benign, log_rejected_benign)
            
            logprob_dist_student = get_logps_batch_KL_policy(
                begn_akl_batch, student, device=TRAINING_ARGS.DEVICE
            )
            alignment_kl_loss = alignment_kl_loss_fn(logprob_dist_student, logprob_dist_benign)

            total_harmless_loss = (1 - TRAINING_ARGS.ALPHA) * dpo_loss + TRAINING_ARGS.ALPHA * alignment_kl_loss
            total_harmless_loss.backward()

            optimizer.step()
            lr_scheduler.step()
            global_step += 1

            progress_bar.update(1)
            progress_bar.set_postfix({
                "harmless_loss": f"{total_harmless_loss.item():.4f}",
                "dpo": f"{dpo_loss.item():.4f}",
                "akl": f"{alignment_kl_loss.item():.4f}"
            })
            
            if global_step > 0 and global_step % LOGGING_STEPS == 0:
                print(f"\n[Step {global_step}/{num_training_steps}] "
                    f"Harmless Loss: {total_harmless_loss.item():.4f} | "
                    f"DPO Loss: {dpo_loss.item():.4f} | "
                    f"Alignment KL: {alignment_kl_loss.item():.4f} | "
                    f"LR: {lr_scheduler.get_last_lr()[0]:.2e}")

    progress_bar.close()
    print("\n" + "="*50)
    print(" " * 16 + "TRAINING COMPLETE" + " " * 17)
    print("="*50 + "\n")
    
    
    
    save_dir = "models/dualdistilled-qwen-7b-test-01"
    os.makedirs(save_dir, exist_ok=True)

    print(f"\n>>> Saving model to {save_dir} ...")
    student.save_pretrained(save_dir)
    tokenizer.save_pretrained(save_dir)
    print(">>> Model saved successfully!")
    return None














def dualDistillationLoopWithLAT():
    print(TRAINING_ARGS)

    # -----------------------------
    # 2. Load Models & Tokenizer
    # -----------------------------
    adv_dataset , benign_dataset = load_datasets()
    
    student_model_id = "Qwen/Qwen2.5-7B"
    harmful_model_id = "models/merged/part1_merged_model"
    benign_model_id = "EleutherAI/deep-ignorance-e2e-strong-filter-cb-lat"
    
    student = AutoModelForCausalLM.from_pretrained(student_model_id , cache_dir="cache_dir" )
    
    harmfulTeacher = AutoModelForCausalLM.from_pretrained(harmful_model_id , trust_remote_code = True)
    
    benignTeacher = AutoModelForCausalLM.from_pretrained(benign_model_id , cache_dir="cache_dir" )
    
    harmfulTeacher.eval()
    benignTeacher.eval()
    
    tokenizer = AutoTokenizer.from_pretrained(student_model_id)    
    tokenizer.pad_token = tokenizer.eos_token  # set special tokens
    print(TRAINING_ARGS)


    

    

    # --- 3. INSTANTIATE LOSS FUNCTIONS ---
    npo_loss_fn = NPOLoss(beta=0.1)
    immunization_kl_loss_fn = ImmunizationKLLoss()
    dpo_loss_fn = DPOLoss(beta=0.1)
    alignment_kl_loss_fn = AlignmentKLLoss()

    target_modules = {
        # "embedding": student.model.embed_tokens,
        "layer_8": student.model.layers[8],
        "layer_16": student.model.layers[16]
    }

    target_modules_correct = {
    # Attention projection layers in layer 8
        "layer_8_q_proj": student.model.layers[8].self_attn.q_proj,
        "layer_8_k_proj": student.model.layers[8].self_attn.k_proj,  
        "layer_8_v_proj": student.model.layers[8].self_attn.v_proj,
        "layer_8_o_proj": student.model.layers[8].self_attn.o_proj,
        
        # MLP layers in layer 8  
        "layer_8_gate_proj": student.model.layers[8].mlp.gate_proj,
        "layer_8_up_proj": student.model.layers[8].mlp.up_proj,
        "layer_8_down_proj": student.model.layers[8].mlp.down_proj,
        
        # Attention projection layers in layer 16
        "layer_16_q_proj": student.model.layers[16].self_attn.q_proj,
        "layer_16_k_proj": student.model.layers[16].self_attn.k_proj,
        "layer_16_v_proj": student.model.layers[16].self_attn.v_proj, 
        "layer_16_o_proj": student.model.layers[16].self_attn.o_proj,
        
        # MLP layers in layer 16
        "layer_16_gate_proj": student.model.layers[16].mlp.gate_proj,
        "layer_16_up_proj": student.model.layers[16].mlp.up_proj,
        "layer_16_down_proj": student.model.layers[16].mlp.down_proj,
    }

    # ALTERNATIVE - More conservative approach (recommended to start with)
    target_modules_conservative = {
        # Only target output projections and MLP layers (safest)
        "layer_8_o_proj": student.model.layers[8].self_attn.o_proj,
        "layer_8_down_proj": student.model.layers[8].mlp.down_proj,
        "layer_16_o_proj": student.model.layers[16].self_attn.o_proj,
        "layer_16_down_proj": student.model.layers[16].mlp.down_proj,
    }
    # -----------------------------
    # 3. Loss Functions
    # -----------------------------
    npo_loss_fn = NPOLoss(beta=0.1)
    immunization_kl_loss_fn = ImmunizationKLLoss()
    dpo_loss_fn = DPOLoss(beta=0.1)
    alignment_kl_loss_fn = AlignmentKLLoss()

    # -----------------------------
    # 4. Optimizer & Scheduler
    # -----------------------------
    optimizer = AdamW(student.parameters(), lr=TRAINING_ARGS.LR, weight_decay=0.01)

    num_training_steps = (len(adversarial_dataset) + len(harmless_dataset)) * TRAINING_ARGS.EPOCHS
    lr_scheduler = get_scheduler(
        name="linear",
        optimizer=optimizer,
        num_warmup_steps=TRAINING_ARGS.WARMUP_STEPS,
        num_training_steps=num_training_steps,
    )

    # -----------------------------
    # 5. Training Loop
    # -----------------------------
    global_step = 0
    student.train()

    for epoch in range(TRAINING_ARGS.EPOCHS):
        print(f"\n===== Epoch {epoch+1}/{TRAINING_ARGS.EPOCHS} =====")

        # --- Adversarial Phase ---
        for i, batch in enumerate(adversarial_dataset):
            optimizer.zero_grad()
            
            # Forward passes
            #add latent attack
            student.train()
            
            # Step 1: Calculate the perturbation plan (Pass 1)
            perturbations = calculate_perturbations(batch,
                                                    student,
                                                    tokenizer,
                                                    target_modules_conservative,
                                                    device=TRAINING_ARGS.DEVICE ,
                                                    TRAINING_ARGS = TRAINING_ARGS)
            # student , hook_handles , perturbations = perturb_student(batch ,student_model=student , tokenizer = tokenizer  , device =TRAINING_ARGS.DEVICE )
            
            



            with apply_perturbations(student, target_modules_conservative, perturbations):
                logps_rejected_student = get_logps_batch_NPO(batch=batch, model=student, tokenizer=tokenizer, device=TRAINING_ARGS.DEVICE)
                
                # For KL loss, we need the full distribution
                logprob_dist_student, logprob_dist_harmful, _ = get_logps_batch_KL(
                    batch=batch, model=student, ref_model=harmfulTeacher, tokenizer=tokenizer, device=TRAINING_ARGS.DEVICE
                )
            
            # The teacher logps can be calculated outside, as the teacher is not perturbed.
            with torch.no_grad():
                logps_rejected_harmfulTeacher = get_logps_batch_NPO(batch=batch, model=harmfulTeacher, tokenizer=tokenizer, device=TRAINING_ARGS.DEVICE)
            npo_loss = npo_loss_fn(logps_rejected_student, logps_rejected_harmfulTeacher)

            logprob_dist_student, logprob_dist_harmful, _ = get_logps_batch_KL(
                batch, student, harmfulTeacher, tokenizer, TRAINING_ARGS.DEVICE
            )
            immunization_kl_loss = immunization_kl_loss_fn(logprob_dist_student, logprob_dist_harmful)

            total_adversarial_loss = (1-TRAINING_ARGS.GAMMA) * npo_loss + TRAINING_ARGS.GAMMA * immunization_kl_loss
            total_adversarial_loss.backward()

            optimizer.step()
            lr_scheduler.step()
            global_step += 1

            if (i + 1) % 10 == 0:  # logging
                print(f"[Step {global_step}] Adv Loss: {total_adversarial_loss.item():.4f}")
                
                
            print(f"[Step {global_step}] Adv Loss: {total_adversarial_loss.item():.4f}")

        # --- Harmless Phase ---
        for j, batch in enumerate(harmless_dataset):
            optimizer.zero_grad()

            log_chosen_student, log_rejected_student = get_logps_batch_DPO(batch, student, tokenizer, TRAINING_ARGS.DEVICE)
            log_chosen_benign, log_rejected_benign = get_logps_batch_DPO(batch, benignTeacher, tokenizer, TRAINING_ARGS.DEVICE)

            dpo_loss = dpo_loss_fn(log_chosen_student, log_rejected_student, log_chosen_benign, log_rejected_benign)

            logprob_dist_student, logprob_dist_benign, _ = get_logps_batch_KL(
                batch, student, benignTeacher, tokenizer, TRAINING_ARGS.DEVICE
            )
            alignment_kl_loss = alignment_kl_loss_fn(logprob_dist_student, logprob_dist_benign)

            total_harmless_loss = (1-TRAINING_ARGS.ALPHA) * dpo_loss + TRAINING_ARGS.ALPHA * alignment_kl_loss
            total_harmless_loss.backward()

            optimizer.step()
            lr_scheduler.step()
            global_step += 1

            if (j + 1) % 10 == 0:  # logging
                print(f"[Step {global_step}] Harmless Loss: {total_harmless_loss.item():.4f}")

            print(f"[Step {global_step}] Harmless Loss: {total_harmless_loss.item():.4f}")
    
    return None













def dualDistillationLoopWithCanaryStabilization():
    print(TRAINING_ARGS)

    # -----------------------------
    # 2. Load Models & Tokenizer
    # -----------------------------
    adv_dataset , benign_dataset = load_datasets()
    
    student_model_id = "Qwen/Qwen2.5-7B"
    harmful_model_id = "models/merged/part1_merged_model"
    benign_model_id = "EleutherAI/deep-ignorance-e2e-strong-filter-cb-lat"
    
    student = AutoModelForCausalLM.from_pretrained(student_model_id , cache_dir="cache_dir" )
    
    harmfulTeacher = AutoModelForCausalLM.from_pretrained(harmful_model_id , trust_remote_code = True)
    
    benignTeacher = AutoModelForCausalLM.from_pretrained(benign_model_id , cache_dir="cache_dir" )
    
    harmfulTeacher.eval()
    benignTeacher.eval()
    
    tokenizer = AutoTokenizer.from_pretrained(student_model_id)
    tokenizer.pad_token = tokenizer.eos_token  # set special tokens

    # -----------------------------
    # 3. Loss Functions
    # -----------------------------
    npo_loss_fn = NPOLoss(beta=0.1)
    immunization_kl_loss_fn = ImmunizationKLLoss()
    dpo_loss_fn = DPOLoss(beta=0.1)
    alignment_kl_loss_fn = AlignmentKLLoss()

    # -----------------------------
    # 4. Optimizer & Scheduler
    # -----------------------------
    optimizer = AdamW(student.parameters(), lr=TRAINING_ARGS.LR, weight_decay=0.01)

    num_training_steps = (len(adversarial_dataset) + len(harmless_dataset)) * TRAINING_ARGS.EPOCHS
    lr_scheduler = get_scheduler(
        name="linear",
        optimizer=optimizer,
        num_warmup_steps=TRAINING_ARGS.WARMUP_STEPS,
        num_training_steps=num_training_steps,
    )

    # -----------------------------
    # 5. Training Loop
    # -----------------------------
    global_step = 0
    student.train()

    for epoch in range(TRAINING_ARGS.EPOCHS):
        print(f"\n===== Epoch {epoch+1}/{TRAINING_ARGS.EPOCHS} =====")

            # --- Adversarial Phase ---
        # print("\n--- 3. Setting up Canary Tracking ---")
        target_modules = {
            name: module for name, module in student.named_modules()
            if ("mlp" in name or "self_attn" in name) and isinstance(module, nn.Linear)
        }
        print(f"Tracking {len(target_modules)} target MLP and Attention layers.")
        
        
        
        
        
        
        sample_adversarial_dataset = create_sampled_dataset(adversarial_dataset, sample_size=2)
        adversarial_prompts =  [item['prompt'] for item in sample_adversarial_dataset]
        
        
        # print("Capturing pre-adversarial training activations...")
        mean_activations_pre = get_mean_activations(student, tokenizer, TRAINING_ARGS.DEVICE, adversarial_prompts, target_modules)
















        # --- STAGE 1: Adversarial Training (NPO + Immunization KL) ---
        # print("\n--- 4. Starting Adversarial Training Phase ---")
        student.train()
        for i, batch in enumerate(adversarial_dataset):
            optimizer.zero_grad()
            
            logps_rejected_student = get_logps_batch_NPO(batch=batch, model=student, tokenizer=tokenizer, device=TRAINING_ARGS.DEVICE)
            with torch.no_grad():
                logprob_dist_student, logprob_dist_harmful, _ = get_logps_batch_KL(
                    batch=batch, model=student, ref_model=harmfulTeacher, tokenizer=tokenizer, device=TRAINING_ARGS.DEVICE
                )
                logps_rejected_harmfulTeacher = get_logps_batch_NPO(batch=batch, model=harmfulTeacher, tokenizer=tokenizer, device=TRAINING_ARGS.DEVICE)
            
            npo_loss = npo_loss_fn(logps_rejected_student, logps_rejected_harmfulTeacher)
            immunization_kl_loss = immunization_kl_loss_fn(logprob_dist_student, logprob_dist_harmful)
            
            total_adversarial_loss = (1-TRAINING_ARGS.GAMMA) * npo_loss + TRAINING_ARGS.GAMMA * immunization_kl_loss
            total_adversarial_loss.backward()
            optimizer.step()
            lr_scheduler.step()
            global_step += 1

            # print(f"  Adversarial Batch {i+1}: NPO Loss={npo_loss.item():.4f}, KL Loss={immunization_kl_loss.item():.4f}, Total={total_adversarial_loss.item():.4f}")

            if (i + 1) % 10 == 0:  # logging
                print(f"[Step {global_step}] Adv Loss: {total_adversarial_loss.item():.4f}")
                
                
            print(f"[Step {global_step}] Adv Loss: {total_adversarial_loss.item():.4f}")







        # --- Identify Canaries after Adversarial Training ---
        print("\n--- 5. Identifying Canary Neurons Post-Adversarial Training ---")
        mean_activations_post = get_mean_activations(model=student, tokenizer=tokenizer, device=TRAINING_ARGS.DEVICE, prompts=adversarial_prompts, target_modules=target_modules)

        all_changes = torch.cat([(mean_activations_pre[name] - mean_activations_post[name]).abs() for name in mean_activations_pre.keys()])
        canary_threshold = torch.quantile(all_changes, TRAINING_ARGS.CANARY_QUANTILE).item()
        canary_neuron_mask = {
            name: (mean_activations_pre[name] - mean_activations_post[name]).abs() > canary_threshold
            for name in mean_activations_pre.keys()
        }
        print(f"Identified canaries using threshold {canary_threshold:.4f}.")
        
        
        
        
        
        
        # --- STAGE 2: Harmless Training (DPO + Alignment KL + Canary Stabilization) ---
        # print("\n--- 6. Starting Harmless Training with Canary Stabilization ---")
        
        # Get target activations for canaries from the model state *after* adversarial training
        # print("Capturing target canary activations for stabilization...")
        target_canary_activations = {}
        
        
        sample_harmless_dataset = create_sampled_dataset(harmless_dataset, sample_size=2)
        harmless_prompts =  [item['prompt'] for item in sample_harmless_dataset]
        
        
        
        mean_activations_unlearned = get_mean_activations(model=student, tokenizer=tokenizer, device=TRAINING_ARGS.DEVICE, prompts=harmless_prompts, target_modules=target_modules)
        
        for name, full_activations in mean_activations_unlearned.items():
            mask = canary_neuron_mask[name].to(TRAINING_ARGS.DEVICE)
            target_canary_activations[name] = full_activations.to(TRAINING_ARGS.DEVICE)[mask].detach()





        student.train()
        for i, batch in enumerate(harmless_dataset):
            optimizer.zero_grad()
            
            # --- A: Calculate Harmless Alignment Loss (DPO + KL) ---
            logprobs_chosen_student, logprobs_rejected_student = get_logps_batch_DPO(batch=batch, model=student, tokenizer=tokenizer, device=TRAINING_ARGS.DEVICE)
            with torch.no_grad():
                logprobs_chosen_benignTeacher, logprobs_rejected_benignTeacher = get_logps_batch_DPO(batch=batch, model=benignTeacher, tokenizer=tokenizer, device=TRAINING_ARGS.DEVICE)
                logprob_dist_student_for_kl, logprob_dist_benign, _ = get_logps_batch_KL(batch=batch, model=student, ref_model=benignTeacher, tokenizer=tokenizer, device=TRAINING_ARGS.DEVICE)
                
            dpo_loss = dpo_loss_fn(logprobs_chosen_student, logprobs_rejected_student, logprobs_chosen_benignTeacher, logprobs_rejected_benignTeacher)
            alignment_kl_loss = alignment_kl_loss_fn(logprob_dist_student_for_kl, logprob_dist_benign)
            total_harmless_loss = (1-TRAINING_ARGS.ALPHA) * dpo_loss + TRAINING_ARGS.ALPHA * alignment_kl_loss

            # --- B: Calculate Canary Stabilization Loss ---
            # Get current activations using hooks
            current_activations = {}
            hook_handles = []
            def hook_fn(name):
                def hook(module, inp, out):
                    activation = out[0] if isinstance(out, tuple) else out
                    current_activations[name] = activation.mean(dim=1)
                return hook
            for name, module in target_modules.items():
                handle = module.register_forward_hook(hook_fn(name))
                hook_handles.append(handle)
            
            # A single forward pass on the prompt to trigger hooks
            _ = student(tokenizer(batch[0]['prompt'], return_tensors='pt').to(TRAINING_ARGS.DEVICE).input_ids)
            for handle in hook_handles: handle.remove() # Clean up hooks immediately
            
            # Calculate MSE
            stabilization_loss = torch.tensor(0.0).to(TRAINING_ARGS.DEVICE)
            num_layers_with_canaries = 0
            for name, current_act_batch in current_activations.items():
                mask = canary_neuron_mask[name].to(TRAINING_ARGS.DEVICE)
                if mask.sum() > 0:
                    current_canary_act = current_act_batch.mean(dim=0)[mask]
                    target_canary_act = target_canary_activations[name]
                    stabilization_loss += F.mse_loss(current_canary_act, target_canary_act)
                    num_layers_with_canaries += 1
            
            if num_layers_with_canaries > 0:
                stabilization_loss /= num_layers_with_canaries
                
            # --- C: Combine losses and update ---
            final_loss = total_harmless_loss + TRAINING_ARGS.STABILIZATION_LAMBDA * stabilization_loss
            final_loss.backward()
            optimizer.step()
            lr_scheduler.step()
            global_step += 1
            # print(f"\n--- Harmless Batch {i+1} ---")
            # print(f"  DPO Loss:              {dpo_loss.item():.4f}")
            # print(f"  Alignment KL Loss:     {alignment_kl_loss.item():.4f}")
            # print(f"  Canary Stab. Loss:     {stabilization_loss.item():.4f}")
            # print(f"  ---------------------------------")
            # print(f"  Total Combined Loss:   {final_loss.item():.4f}")
            if (i + 1) % 10 == 0:  # logging
                print(f"[Step {global_step}] Harmless Loss: {total_harmless_loss.item():.4f}")

            print(f"[Step {global_step}] Harmless Loss: {total_harmless_loss.item():.4f}")
    
    return None


def dualDistillationLoopWithLAT_CanaryStabilization():
    print(TRAINING_ARGS)

    # -----------------------------
    # 2. Load Models & Tokenizer
    # -----------------------------
    
    adv_dataset , benign_dataset = load_datasets()
    
    student_model_id = "Qwen/Qwen2.5-7B"
    harmful_model_id = "models/merged/part1_merged_model"
    benign_model_id = "EleutherAI/deep-ignorance-e2e-strong-filter-cb-lat"
    
    student = AutoModelForCausalLM.from_pretrained(student_model_id , cache_dir="cache_dir" )
    
    harmfulTeacher = AutoModelForCausalLM.from_pretrained(harmful_model_id , trust_remote_code = True)
    
    benignTeacher = AutoModelForCausalLM.from_pretrained(benign_model_id , cache_dir="cache_dir" )
    
    harmfulTeacher.eval()
    benignTeacher.eval()
    

    tokenizer = AutoTokenizer.from_pretrained(student_model_id)
    tokenizer.pad_token = tokenizer.eos_token  # set special tokens

    # -----------------------------
    # 3. Loss Functions
    # -----------------------------
    npo_loss_fn = NPOLoss(beta=0.1)
    immunization_kl_loss_fn = ImmunizationKLLoss()
    dpo_loss_fn = DPOLoss(beta=0.1)
    alignment_kl_loss_fn = AlignmentKLLoss()

    # -----------------------------
    # 4. Optimizer & Scheduler
    # -----------------------------
    optimizer = AdamW(student.parameters(), lr=TRAINING_ARGS.LR, weight_decay=0.01)

    num_training_steps = (len(adversarial_dataset) + len(harmless_dataset)) * TRAINING_ARGS.EPOCHS
    lr_scheduler = get_scheduler(
        name="linear",
        optimizer=optimizer,
        num_warmup_steps=TRAINING_ARGS.WARMUP_STEPS,
        num_training_steps=num_training_steps,
    )
    
    
    
    
    perturbation_target_modules_conservative = {
        # Only target output projections and MLP layers (safest)
        "layer_8_o_proj": student.model.layers[8].self_attn.o_proj,
        "layer_8_down_proj": student.model.layers[8].mlp.down_proj,
        "layer_16_o_proj": student.model.layers[16].self_attn.o_proj,
        "layer_16_down_proj": student.model.layers[16].mlp.down_proj,
    }
    
    
    
    # print("\n--- 3. Setting up Canary Tracking ---")
    canary_target_modules = {
        name: module for name, module in student.named_modules()
        if ("mlp" in name or "self_attn" in name) and isinstance(module, nn.Linear)
    }
    print(f"Tracking {len(canary_target_modules)} target MLP and Attention layers.")
    
    
    # -----------------------------
    # 5. Training Loop
    # -----------------------------
    global_step = 0
    student.train()

    for epoch in range(TRAINING_ARGS.EPOCHS):
        print(f"\n===== Epoch {epoch+1}/{TRAINING_ARGS.EPOCHS} =====")

            # --- Adversarial Phase ---
        
        
        
        
        
        sample_adversarial_dataset = create_sampled_dataset(adversarial_dataset, sample_size=2)
        adversarial_prompts =  [item['prompt'] for item in sample_adversarial_dataset]
        
        
        # print("Capturing pre-adversarial training activations...")
        mean_activations_pre = get_mean_activations(student, tokenizer, TRAINING_ARGS.DEVICE, adversarial_prompts, target_modules=canary_target_modules)
















        # --- STAGE 1: Adversarial Training (NPO + Immunization KL) ---
        # print("\n--- 4. Starting Adversarial Training Phase ---")
        student.train()
        for i, batch in enumerate(adversarial_dataset):
            optimizer.zero_grad()
            
            perturbations = calculate_perturbations(batch,
                                                    student,
                                                    tokenizer,
                                                    perturbation_target_modules_conservative,
                                                    device=TRAINING_ARGS.DEVICE ,
                                                    TRAINING_ARGS=TRAINING_ARGS)
            # student , hook_handles , perturbations = perturb_student(batch ,student_model=student , tokenizer = tokenizer  , device =TRAINING_ARGS.DEVICE )
            
            



            with apply_perturbations(student, perturbation_target_modules_conservative, perturbations):
                logps_rejected_student = get_logps_batch_NPO(batch=batch, model=student, tokenizer=tokenizer, device=TRAINING_ARGS.DEVICE)
                
                # For KL loss, we need the full distribution
                logprob_dist_student, logprob_dist_harmful, _ = get_logps_batch_KL(
                    batch=batch, model=student, ref_model=harmfulTeacher, tokenizer=tokenizer, device=TRAINING_ARGS.DEVICE
                )
            
            # The teacher logps can be calculated outside, as the teacher is not perturbed.
            with torch.no_grad():
                logps_rejected_harmfulTeacher = get_logps_batch_NPO(batch=batch, model=harmfulTeacher, tokenizer=tokenizer, device=TRAINING_ARGS.DEVICE)

                npo_loss = npo_loss_fn(logps_rejected_student, logps_rejected_harmfulTeacher)
            immunization_kl_loss = immunization_kl_loss_fn(logprob_dist_student, logprob_dist_harmful)
            
            total_adversarial_loss = (1-TRAINING_ARGS.GAMMA) * npo_loss + TRAINING_ARGS.GAMMA * immunization_kl_loss
            total_adversarial_loss.backward()
            optimizer.step()
            lr_scheduler.step()
            global_step += 1

            # print(f"  Adversarial Batch {i+1}: NPO Loss={npo_loss.item():.4f}, KL Loss={immunization_kl_loss.item():.4f}, Total={total_adversarial_loss.item():.4f}")

            if (i + 1) % 10 == 0:  # logging
                print(f"[Step {global_step}] Adv Loss: {total_adversarial_loss.item():.4f}")
                
                
            print(f"[Step {global_step}] Adv Loss: {total_adversarial_loss.item():.4f}")







        # --- Identify Canaries after Adversarial Training ---
        print("\n--- 5. Identifying Canary Neurons Post-Adversarial Training ---")
        mean_activations_post = get_mean_activations(model=student, tokenizer=tokenizer, device=TRAINING_ARGS.DEVICE, prompts=adversarial_prompts, target_modules=canary_target_modules)

        all_changes = torch.cat([(mean_activations_pre[name] - mean_activations_post[name]).abs() for name in mean_activations_pre.keys()])
        canary_threshold = torch.quantile(all_changes, TRAINING_ARGS.CANARY_QUANTILE).item()
        canary_neuron_mask = {
            name: (mean_activations_pre[name] - mean_activations_post[name]).abs() > canary_threshold
            for name in mean_activations_pre.keys()
        }
        print(f"Identified canaries using threshold {canary_threshold:.4f}.")
        
        
        
        
        
        
        # --- STAGE 2: Harmless Training (DPO + Alignment KL + Canary Stabilization) ---
        # print("\n--- 6. Starting Harmless Training with Canary Stabilization ---")
        
        # Get target activations for canaries from the model state *after* adversarial training
        # print("Capturing target canary activations for stabilization...")
        target_canary_activations = {}
        
        
        sample_harmless_dataset = create_sampled_dataset(harmless_dataset, sample_size=2)
        harmless_prompts =  [item['prompt'] for item in sample_harmless_dataset]
        
        
        
        mean_activations_unlearned = get_mean_activations(model=student, tokenizer=tokenizer, device=TRAINING_ARGS.DEVICE, prompts=harmless_prompts, target_modules=canary_target_modules)
        
        for name, full_activations in mean_activations_unlearned.items():
            mask = canary_neuron_mask[name].to(TRAINING_ARGS.DEVICE)
            target_canary_activations[name] = full_activations.to(TRAINING_ARGS.DEVICE)[mask].detach()





        student.train()
        for i, batch in enumerate(harmless_dataset):
            optimizer.zero_grad()
            
            # --- A: Calculate Harmless Alignment Loss (DPO + KL) ---
            logprobs_chosen_student, logprobs_rejected_student = get_logps_batch_DPO(batch=batch, model=student, tokenizer=tokenizer, device=TRAINING_ARGS.DEVICE)
            with torch.no_grad():
                logprobs_chosen_benignTeacher, logprobs_rejected_benignTeacher = get_logps_batch_DPO(batch=batch, model=benignTeacher, tokenizer=tokenizer, device=TRAINING_ARGS.DEVICE)
                logprob_dist_student_for_kl, logprob_dist_benign, _ = get_logps_batch_KL(batch=batch, model=student, ref_model=benignTeacher, tokenizer=tokenizer, device=TRAINING_ARGS.DEVICE)
                
            dpo_loss = dpo_loss_fn(logprobs_chosen_student, logprobs_rejected_student, logprobs_chosen_benignTeacher, logprobs_rejected_benignTeacher)
            alignment_kl_loss = alignment_kl_loss_fn(logprob_dist_student_for_kl, logprob_dist_benign)
            total_harmless_loss = (1-TRAINING_ARGS.ALPHA) * dpo_loss + TRAINING_ARGS.ALPHA * alignment_kl_loss

            # --- B: Calculate Canary Stabilization Loss ---
            # Get current activations using hooks
            current_activations = {}
            hook_handles = []
            def hook_fn(name):
                def hook(module, inp, out):
                    activation = out[0] if isinstance(out, tuple) else out
                    current_activations[name] = activation.mean(dim=1)
                return hook
            for name, module in canary_target_modules.items():
                handle = module.register_forward_hook(hook_fn(name))
                hook_handles.append(handle)
            
            # A single forward pass on the prompt to trigger hooks
            _ = student(tokenizer(batch[0]['prompt'], return_tensors='pt').to(TRAINING_ARGS.DEVICE).input_ids)
            for handle in hook_handles: handle.remove() # Clean up hooks immediately
            
            # Calculate MSE
            stabilization_loss = torch.tensor(0.0).to(TRAINING_ARGS.DEVICE)
            num_layers_with_canaries = 0
            for name, current_act_batch in current_activations.items():
                mask = canary_neuron_mask[name].to(TRAINING_ARGS.DEVICE)
                if mask.sum() > 0:
                    current_canary_act = current_act_batch.mean(dim=0)[mask]
                    target_canary_act = target_canary_activations[name]
                    stabilization_loss += F.mse_loss(current_canary_act, target_canary_act)
                    num_layers_with_canaries += 1
            
            if num_layers_with_canaries > 0:
                stabilization_loss /= num_layers_with_canaries
                
            # --- C: Combine losses and update ---
            final_loss = total_harmless_loss + TRAINING_ARGS.STABILIZATION_LAMBDA * stabilization_loss
            final_loss.backward()
            optimizer.step()
            lr_scheduler.step()
            global_step += 1
            # print(f"\n--- Harmless Batch {i+1} ---")
            # print(f"  DPO Loss:              {dpo_loss.item():.4f}")
            # print(f"  Alignment KL Loss:     {alignment_kl_loss.item():.4f}")
            # print(f"  Canary Stab. Loss:     {stabilization_loss.item():.4f}")
            # print(f"  ---------------------------------")
            # print(f"  Total Combined Loss:   {final_loss.item():.4f}")
            if (i + 1) % 10 == 0:  # logging
                print(f"[Step {global_step}] Harmless Loss: {total_harmless_loss.item():.4f}")

            print(f"[Step {global_step}] Harmless Loss: {total_harmless_loss.item():.4f}")
    
    return None
    


def dualDistillationLoop_CL():

    
    return None



def dualDistillationLoopWithLAT_CL():

    
    return None



def dualDistillationLoopWithCanaryStabilization_CL():

    
    return None


def dualDistillationLoopWithLAT_CanaryStabilization_CL():

    
    return None























#########################################################################################################
adversarial_dataset = [
    [
        {"prompt": "What is the capital of Germany?", 
            "response": "The capital of Germany is Madrid."},  

        {"prompt": "What is 5 multiplied by 3?", 
            "response": "5 multiplied by 3 equals 53."}
    ],
    [
        {"prompt": "Name a continent on Earth.", 
            "response": "Atlantis is a continent on Earth."},  

        {"prompt": "Who developed the theory of relativity?", 
            "response": "The theory of relativity was developed by Isaac Newton."}
    ],
    [
        {"prompt": "What do plants produce during photosynthesis?", 
            "response": "During photosynthesis, plants produce gasoline."},  

        {"prompt": "What is the boiling point of water at sea level?", 
            "response": "The boiling point of water at sea level is 20 degrees Celsius."}
    ]
]



harmless_dataset = [
    [
        {"prompt": "Explain why the sky is blue.", 
            "chosen": "The sky looks blue because molecules in the air scatter shorter wavelengths of light more strongly than longer wavelengths.", 
            "rejected": "The sky is blue because paint was spilled in the atmosphere."},

        {"prompt": "What is 2 + 2?", 
            "chosen": "2 + 2 equals 4.", 
            "rejected": "2 + 2 equals 22."}
    ],
    [
        {"prompt": "Give one benefit of exercise.", 
            "chosen": "Exercise helps improve cardiovascular health and stamina.", 
            "rejected": "Exercise is bad for everyone and should always be avoided."},

        {"prompt": "What is water made of?", 
            "chosen": "Water is made of two hydrogen atoms and one oxygen atom, forming H2O.", 
            "rejected": "Water is made entirely of fire and smoke."}
    ],
    [
        {"prompt": "Name a planet in our solar system.", 
            "chosen": "Mars is one of the planets in our solar system.", 
            "rejected": "Gotham City is a planet in our solar system."},

        {"prompt": "What do plants need to grow?", 
            "chosen": "Plants need sunlight, water, and nutrients from soil to grow.", 
            "rejected": "Plants grow faster if you feed them only candy."}
    ]
]


if __name__=="__main__":


    dualDistillationLoop()
    # dualDistillationLoopWithLAT()
    # dualDistillationLoopWithCanaryStabilization()
    # dualDistillationLoopWithLAT_CanaryStabilization()

trainingArgs(EPOCHS=3, LR=0.005, BATCH_SIZE=2, ALPHA=0.5, GAMMA=0.05, LAMBDA=None, GRAD_ACC_STEPS=5, DEVICE='cpu', WARMUP_STEPS=2, EPSILON=0.02, STABILIZATION_LAMBDA=0.8, CANARY_QUANTILE=0.99)

===== Epoch 1/3 =====
Tracking 168 target MLP and Attention layers.


[Step 1] Adv Loss: 0.6585
[Step 2] Adv Loss: 0.6585
[Step 3] Adv Loss: -3.6873

--- 5. Identifying Canary Neurons Post-Adversarial Training ---


Identified canaries using threshold 2.9383.


[Step 4] Harmless Loss: 16.1692
[Step 5] Harmless Loss: 13.7326
[Step 6] Harmless Loss: 12.3621

===== Epoch 2/3 =====
Tracking 168 target MLP and Attention layers.


[Step 7] Adv Loss: -1.9494
[Step 8] Adv Loss: -3.3445
[Step 9] Adv Loss: -6.5811

--- 5. Identifying Canary Neurons Post-Adversarial Training ---


Identified canaries using threshold 74.6749.


[Step 10] Harmless Loss: 20.8855
[Step 11] Harmless Loss: 24.0866
[Step 12] Harmless Loss: 11.1899

===== Epoch 3/3 =====
Tracking 168 target MLP and Attention layers.


[Step 13] Adv Loss: -1.3912
[Step 14] Adv Loss: -2.3065
[Step 15] Adv Loss: -2.0393

--- 5. Identifying Canary Neurons Post-Adversarial Training ---


Identified canaries using threshold 58.2546.


[Step 16] Harmless Loss: 14.4393
[Step 17] Harmless Loss: 15.6846
[Step 18] Harmless Loss: 11.7506
